In this notebook, we will try to focus in the US dynamic to be able to determine the virus mobility as well as it's R factor depending on different local factors. The use of this notebook is diagnosis on the effectivness of certain measures on the R factor as well as. 

In [ ]:
#Los clasicos
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # para leer datos
import sklearn.manifold
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state
from sklearn.metrics import f1_score
#Ejercicio 16
import sklearn.metrics
import sklearn.tree
#Ejercicio 24
from sklearn.linear_model import LogisticRegression
import sklearn.neural_network
#Ejercicio 20, clusters umap
import sklearn.cluster

#Puede ser interesante el ejercicio 8 para fitear usando bayes, 7 con MCMH .
#No se me puede olvidar la de precondiciones del cdc

In [ ]:
folderset = [folder for folder in glob.glob("Uncover/" + "**/", recursive=True)]
